## Data Loaders

This notebook demonstrates the use of edge loader in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.

Note: For the data loaders to work, a few UDFs (User Defined Functions) have to be installed into the TigerGraph database.

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its documentation for details.

In [2]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    useCert=False
)

In [3]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# Number of edges for every type
conn.getEdgeCount()

{'Cite': 10556}

### Edge Loader

EdgeLoader pulls batches of edges from database. Specifically, it divides edges into `num_batches` and returns each batch separately. The boolean attribute provided to `filter_by` indicates which edges are included. If you need random batches, set `shuffle` to True.

**Note**: For the first time you initialize the loader on a graph in TigerGraph,
the initialization might take a minute as it installs the corresponding
query to the database and optimizes it. However, the query installation only
needs to be done once, so it will take no time when you initialize the loader
on the same TG graph again.

There are two ways to use the data loader. See
[here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all edges at once (`num_batches=1`), there will be only one batch (of all the edges) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader again.

Args:
* attributes (list or dict, optional):
        Edge attributes to be included. If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type. Defaults to None.
* batch_size (int, optional):  
        Number of edges in each batch.  
        Defaults to None.  
* num_batches (int, optional):  
        Number of batches to split the edges.  
        Defaults to 1.  
* shuffle (bool, optional):  
        Whether to shuffle the edges before loading data.  
        Defaults to False.  
* filter_by (str, optional):
        A boolean attribute used to indicate which edges are included. Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader. Only
        "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* kafka_address (str, optional):
        Address of the kafka broker. Defaults to None.
* kafka_max_msg_size (int, optional):
        Maximum size of a Kafka message in bytes.
        Defaults to 104857600.
* kafka_num_partitions (int, optional):
        Number of partitions for the topic created by this loader.
        Defaults to 1.
* kafka_replica_factor (int, optional):
        Number of replications for the topic created by this
        loader. Defaults to 1.
* kafka_retention_ms (int, optional):
        Retention time for messages in the topic created by this
        loader in milliseconds. Defaults to 60000.
* kafka_auto_del_topic (bool, optional):
        Whether to delete the Kafka topic once the
        loader finishes pulling data. Defaults to True.
* kafka_address_consumer (str, optional):
        Address of the kafka broker that a consumer
        should use. Defaults to be the same as `kafkaAddress`.
* kafka_address_producer (str, optional):
        Address of the kafka broker that a producer
        should use. Defaults to be the same as `kafkaAddress`.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

#### Load all edges at once directly to local. Default.

In [5]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=1,
    attributes=["time", "is_train"])

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 217 ms, sys: 38.5 ms, total: 255 ms
Wall time: 37.5 s


In [6]:
%%time
# Get the only batch of data via the `data` property
data = edge_loader.data
data.shape

CPU times: user 9.4 ms, sys: 4.09 ms, total: 13.5 ms
Wall time: 32.9 ms


(10556, 4)

In [7]:
data.head()

,source,target,time,is_train
0,8388608,1048600,0,1
1,8388608,12582912,0,1
2,8388608,13631492,0,1
3,8388608,31457280,0,1
4,8388609,22020127,0,1


#### Get batches of edges through http

In [8]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=10,
    attributes=["time", "is_train"],
    shuffle=True,
    filter_by=None
)

CPU times: user 3.92 ms, sys: 952 µs, total: 4.87 ms
Wall time: 4.98 ms


In [9]:
%%time
for i, batch in enumerate(edge_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (1129, 4)----
    source    target  time  is_train
0  3145728  22020185     0         1
----Batch 1: Shape (1002, 4)----
    source    target  time  is_train
0  1048577  20971586     0         1
----Batch 2: Shape (1124, 4)----
   source   target  time  is_train
0       4  9437199     0         1
----Batch 3: Shape (1071, 4)----
     source    target  time  is_train
0  11534340  32505859     0         1
----Batch 4: Shape (978, 4)----
     source    target  time  is_train
0  11534341  16777293     0         1
----Batch 5: Shape (1149, 4)----
    source   target  time  is_train
0  5242882  2097158     0         1
----Batch 6: Shape (1013, 4)----
    source    target  time  is_train
0  4194305  23068698     0         1
----Batch 7: Shape (1037, 4)----
    source   target  time  is_train
0  7340035  4194337     0         0
----Batch 8: Shape (1067, 4)----
   source   target  time  is_train
0       3  1048595     0         1
----Batch 9: Shape (986, 4)----
    source    

#### Stream batches of edges through Kafka

In [10]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=10,
    attributes=["time", "is_train"],
    shuffle=True,
    filter_by=None,
    kafka_address="127.0.0.1:9092",
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 45 ms, sys: 5.45 ms, total: 50.4 ms
Wall time: 25.4 s


In [11]:
%%time
for i, batch in enumerate(edge_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (995, 4)----
   source    target  time  is_train
0       1  13631492     0         1
----Batch 1: Shape (1118, 4)----
    source   target  time  is_train
0  8388608  1048600     0         1
----Batch 2: Shape (1130, 4)----
     source   target  time  is_train
0  11534337  7340055     0         1
----Batch 3: Shape (1069, 4)----
    source    target  time  is_train
0  5242889  18874424     0         1
----Batch 4: Shape (1056, 4)----
    source    target  time  is_train
0  3145728  16777284     0         1
----Batch 5: Shape (1037, 4)----
    source   target  time  is_train
0  1048578  2097165     0         1
----Batch 6: Shape (1026, 4)----
     source    target  time  is_train
0  13631490  17825817     0         0
----Batch 7: Shape (1048, 4)----
    source   target  time  is_train
0  4194304  9437199     0         1
----Batch 8: Shape (1083, 4)----
     source    target  time  is_train
0  25165829  31457319     0         0
----Batch 9: Shape (994, 4)----
   source 

#### For heterogeneous graphs

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [3]:
%%time
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="hetero",
    username="tigergraph",
    password="tigergraph",
    useCert=False
)

CPU times: user 87 µs, sys: 14 µs, total: 101 µs
Wall time: 105 µs


In [4]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [5]:
%%time
loader = conn.gds.edgeLoader(
    attributes={"v0v0": ["is_train", "is_val"],
                "v2v0": ["is_train", "is_val"]},
    batch_size=200,
    shuffle=False,
    filter_by=None,
    loader_id=None,
    buffer_size=4
)

CPU times: user 35.3 ms, sys: 7.42 ms, total: 42.8 ms
Wall time: 369 ms


In [9]:
%%time
for i, batch in enumerate(loader):
    print("----Batch {}----".format(i))
    for j in batch:
        print("Edge type:", j)
        print(batch[j].head(1))

----Batch 0----
Edge type: v0v0
     source    target is_train is_val
0  69206018  77594624        0      0
Edge type: v2v0
      source    target is_train is_val
0  138412034  72351746        0      0
----Batch 1----
Edge type: v0v0
     source    target is_train is_val
0  68157440  94371841        0      0
Edge type: v2v0
      source    target is_train is_val
0  139460608  67108869        0      0
----Batch 2----
Edge type: v2v0
      source    target is_train is_val
0  137363457  68157440        0      0
Edge type: v0v0
     source    target is_train is_val
0  67108866  75497473        0      0
----Batch 3----
Edge type: v0v0
     source    target is_train is_val
0  75497474  87031809        0      0
Edge type: v2v0
      source    target is_train is_val
0  108003329  80740352        0      0
----Batch 4----
Edge type: v0v0
     source    target is_train is_val
0  76546048  71303169        0      0
Edge type: v2v0
      source    target is_train is_val
0  134217730  92274688       